In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
!ls "/content/gdrive/MyDrive/data"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 face.csv	        feature_extractByPretrain.npy
 faceData	        model4emotionClassfier
 faceData2D	        model4multiClassfier
 faceData2DWithNumber  'model4multiClassfier(exceptAge)'
 face_Dataset	        model4raceClassfier
 faceDataset	        model4sexClassfier
 faceDataset2	        model4sexClassfierWithoutValid
 faceDataWithNumber     rawdata
 faceDR		        rawdata.zip
 faceDS		        vgg_face_dag.pth
 faceLabel.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from skimage.feature import hog
from sklearn.metrics import accuracy_score

In [4]:
pre_path="/content/gdrive/MyDrive/data/"

In [5]:
face=pd.read_csv(pre_path+"faceLabel.csv")
feature=np.load("/content/gdrive/MyDrive/data/feature_extractByPretrain.npy")

In [6]:
face

,number,sex,age,race,emotion,extra,sex_label,age_adult,age_child,age_senior,...,race_black,race_hispanic,race_other,race_white,emotion_funny,emotion_serious,emotion_smiling,race_label,emotion_label,age_label
0,1223,male,child,white,smiling,NaN,1,0,1,0,...,0,0,0,1,0,0,1,4,2,1
1,1224,male,child,white,serious,NaN,1,0,1,0,...,0,0,0,1,0,1,0,4,1,1
2,1225,male,child,white,smiling,NaN,1,0,1,0,...,0,0,0,1,0,0,1,4,2,1
3,1226,male,child,white,smiling,NaN,1,0,1,0,...,0,0,0,1,0,0,1,4,2,1
4,1227,male,child,white,serious,NaN,1,0,1,0,...,0,0,0,1,0,1,0,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3988,5218,female,adult,white,smiling,NaN,0,1,0,0,...,0,0,0,1,0,0,1,4,2,0
3989,5219,female,adult,white,smiling,NaN,0,1,0,0,...,0,0,0,1,0,0,1,4,2,0
3990,5220,female,adult,white,serious,NaN,0,1,0,0,...,0,0,0,1,0,1,0,4,1,0
3991,5221,female,adult,white,smiling,NaN,0,1,0,0,...,0,0,0,1,0,0,1,4,2,0


In [7]:

X_train, X_test, y_train, y_test = train_test_split(feature,  face['emotion_label'], random_state=0)

In [19]:
adb = AdaBoostClassifier(n_estimators=500, random_state=0)
adb.fit(X_train, y_train)

print(adb.score(X_test, y_test))
# sex n:100 0.7917917917917918
# sex n:500 0.7917917917917918

0.6276276276276276


In [20]:
rfc = RandomForestClassifier(n_estimators=500, random_state=0,verbose=1)
rfc.fit(X_train, y_train)

print(rfc.score(X_test, y_test))
# sex n:200 0.8168168168168168
# sex n:300 0.8218218218218218
# sex n:500 0.8198198198198198

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7217217217217218


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   46.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.2s finished


In [21]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=0)),
    ('ab', AdaBoostClassifier(n_estimators=10, random_state=0))
]

sc = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
sc.fit(X_train, y_train)

print(sc.score(X_test, y_test))

0.6466466466466466


---


In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

gamma_range = np.logspace(-10,1,10)
coef0_range = np.linspace(0,5,10)
C_range = np.linspace(0.01,30,10)
degree_range = np.arange(1,11)
param_grid = dict(gamma = gamma_range
                 ,coef0 = coef0_range
                 ,C = C_range
                 ,degree = degree_range
                 )
cv = KFold(n_splits=5,shuffle=True,random_state=520)
grid = GridSearchCV(SVC(kernel = "poly"),param_grid=param_grid, cv=cv, n_jobs = -1,verbose=2) # n_jobs = -1 调用所有线程
grid.fit(X_train, y_train)
print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, grid.best_score_))

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits


In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

gamma_range = np.logspace(-10,1,10)
coef0_range = np.linspace(0,5,10)
C_range = np.linspace(0.01,30,20)
param_grid = dict(gamma = gamma_range
                 ,coef0 = coef0_range
                 ,C = C_range
                 )
cv = KFold(n_splits=5,shuffle=True,random_state=520)
grid = GridSearchCV(SVC(kernel = "sigmoid"),param_grid=param_grid, cv=cv, n_jobs = -1) # n_jobs = -1 调用所有线程
grid.fit(X_train, y_train)
print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, grid.best_score_))

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

gamma_range = np.logspace(-10,1,10)
C_range = np.linspace(0.01,30,20)
param_grid = dict(gamma = gamma_range
                 ,C = C_range
                 )
cv = KFold(n_splits=5,shuffle=True,random_state=520)
grid = GridSearchCV(SVC(kernel = "rbf"),param_grid=param_grid, cv=cv, n_jobs = -1) # n_jobs = -1 调用所有线程
grid.fit(X_train, y_train)
print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, grid.best_score_))

In [ ]:
C_range = np.linspace(0.01, 30, 30)
best_acc = -1
best_c = -1
for c in C_range:
    clf = SVC(kernel="linear", C=c)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    if acc > best_acc:
        best_acc = acc
        best_c = c
print("The best c is %0.5f with a score of %0.5f" % (best_c, best_acc))

探究各参数的影响

In [ ]:
score = []
gamma_range = np.logspace(-10, 1, 50) #返回在对数刻度上均匀间隔的数字
for i in gamma_range:
    clf = SVC(kernel="rbf", gamma = i, cache_size=5000).fit(X_train,y_train)
    score.append(clf.score(X_test, y_test))
print(max(score), gamma_range[score.index(max(score))])
plt.plot(gamma_range, score)
plt.xlabel('gamma')
plt.ylabel('Accuracy')
plt.savefig('pt1.jpg')
plt.show()

In [ ]:
score = []
C_range = np.linspace(0.01, 30, 30)
for i in C_range:
    clf = SVC(kernel="rbf", C = i, cache_size=5000).fit(X_train,y_train)
    score.append(clf.score(X_test, y_test))
print(max(score), C_range[score.index(max(score))])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.savefig('pt2.jpg')
plt.plot(C_range, score)
plt.show()

和其它方式进行对比

In [8]:
svm = SVC(C = 15.52, kernel='linear',verbose=2)
svm.fit(X_train, y_train)
Y_predict = svm.predict(X_test)


[LibSVM]

NameError: ignored

In [11]:
acc = accuracy_score(y_test, Y_predict)
print('SVM准确率为: ', acc)

SVM准确率为:  0.6986986986986987


In [12]:
knn = KNeighborsClassifier(n_neighbors=1) # k取1，最近邻准确率较高
knn.fit(X_train,y_train)
Y_predict = knn.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('KNN准确率为: ', acc)

KNN准确率为:  0.5735735735735735


In [13]:
tree_D = DecisionTreeClassifier()
tree_D.fit(X_train, y_train)
Y_predict = tree_D.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('决策树准确率为: ', acc)

决策树准确率为:  0.5685685685685685


In [14]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)
Y_predict = logistic.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('逻辑回归准确率为: ', acc)

逻辑回归准确率为:  0.7547547547547547


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
mlt = GaussianNB()
mlt.fit(X_train, y_train)
Y_predict = mlt.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('朴素贝叶斯准确率为: ', acc)

朴素贝叶斯准确率为:  0.5245245245245245


In [16]:
Forest = RandomForestClassifier(n_estimators=180,random_state=0)
Forest.fit(X_train, y_train)
Y_predict = Forest.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('随机森林准确率为: ', acc)

随机森林准确率为:  0.7137137137137137


In [17]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
svc = SVC(C = 20, kernel='poly')
clf = BaggingClassifier(base_estimator=svc, n_estimators=20, max_samples=1.0, max_features=1.0,
                        bootstrap=True,bootstrap_features=False, n_jobs=-1, random_state=1)
clf.fit(X_train, y_train)
Y_predict = clf.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('SVM+Bagging准确率为: ', acc)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


SVM+Bagging准确率为:  0.7267267267267268


In [18]:
from xgboost import XGBClassifier as XGBR
reg = XGBR(n_estimators=200
          ,learning_rate=0.1
          ,booster="gblinear"
          ).fit(X_train,y_train,eval_metric=['logloss','auc','error'])
Y_predict = reg.predict(X_test)
acc = accuracy_score(y_test, Y_predict)
print('XGBoost准确率为: ', acc)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


XGBoost准确率为:  0.7577577577577578
